<a href="https://colab.research.google.com/github/Rojan116/NepaliHandWrittenAlphabetRecognizer/blob/master/my_own_cnn_for_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
x = np.array([[3,3,3],
             [3,2,0],
             [-1,2,-4]])

In [18]:
y = np.maximum(x,0)
y

array([[3, 3, 3],
       [3, 2, 0],
       [0, 2, 0]])

In [0]:
a=[1,4,5]

In [0]:
output_size = 10
def convert_y(a):
    arr = np.zeros((output_size,1))
    arr[a] = 1
    return arr


In [27]:
convert_y(a)

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [0]:
u=np.zeros((output_size,1))

In [17]:
u

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [30]:
import numpy as np
import h5py
import copy
from random import randint

#reading mnist dataset
#load dataset
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
#training set
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
#test set
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ) )
MNIST_data.close()

#number of training set
print(x_train.shape)
#number of test set
print(x_test.shape)
#display size of dataset image
print(x_train[0].shape)
print(x_test[0].shape)


(60000, 784)
(10000, 784)
(784,)
(784,)


In [0]:
total_inputs = len(x_train) #number training set
output_size = 10  #number of class
num_filters=5     #number of filter used in convolution layer
filter_size=3     #size of filter/kernel
input_dim=28      #input dimension
input_size=28*28  #input size

LR = .01          #learning Rate
num_epochs = 10   #Epochs 


In [0]:
#K is N number of n*n  size filter
K = np.random.randn(filter_size,filter_size, num_filters)/ np.sqrt(filter_size)

#initial random weight
W = np.random.randn(output_size,input_dim-filter_size+1,input_dim-filter_size+1,num_filters) / np.sqrt(input_dim-filter_size+1)

#initial bias values
bias= np.zeros((output_size, 1))/np.sqrt(output_size)


In [55]:
#number of filter
print(K.shape)
#number of random weights
print(W.shape)
#number of bias 
print(bias.shape)

(3, 3, 5)
(10, 26, 26, 5)
(10, 1)


In [0]:
#convolutional layer
def conv(x,K):
	#Z=np.zeros((input_dim-filter_size+1,input_dim-filter_size+1,num_filters))
	Z=np.zeros(((x.shape[0]-K.shape[0]+1),(x.shape[0]-K.shape[0]+1),K.shape[2]))
	
	for p in range(K.shape[2]):
		for i in range(Z.shape[0]):
			for j in range(Z.shape[1]):
					if (i+3<Z.shape[0] and j+3<Z.shape[1]):
						x_temp = x[i:i+3,j:j+3]
						temp=np.multiply(x_temp,K[:,:,p])
						Z[i,j,p]=np.sum(temp)
	return Z


In [0]:
#one hot encoding
def convert_y(y):
    arr = np.zeros((output_size,1))  #initialize id array of output size 
    arr[y] = 1       #assign 1 to value of array at index y
    return arr       #return one hot encoded array


In [0]:
#softmax activation function
def softmax_function(z):
    ZZ = np.exp(z - max(z))/np.sum(np.exp(z - max(z)))   
    return ZZ

In [0]:
#relu activation function
def Relu(z):
    return np.maximum(z,0)    #return zero for those value less than zero if x<0 then x=0 else x =x

In [0]:
#gardient relu activation function
def gradient_Relu(z):
    return np.where(z>0,1,0)  #return zero for value less than zero else return 1 

In [61]:
Relu(x)


array([[3, 3, 3],
       [3, 2, 0],
       [0, 2, 0]])

In [75]:
gradient_Relu(x) 

array([[1, 1, 1],
       [1, 1, 0],
       [0, 1, 0]])

In [0]:
x = x_train[n_random][:]

In [78]:
x.shape

(784,)

In [0]:
#fitting model
for epochs in range(num_epochs):   #looping through number of epochs

    if (epochs > 5):               
        LR = 0.001                  #set learning rate 0.001 if epohs >5 
    if (epochs > 10):
        LR = 0.0001                 #set learning rate 0.0001 if epohs >10
    if (epochs > 15):
        LR = 0.00001                #set learning rate 0.00001 if epohs >15 

    total_correct = 0               #initialize count variable to store correct count

    for n in range(len(x_train)):    #looping through number of training image
        n_random = randint(0,len(x_train)-1)  #shuffel dataset randomly
        y = y_train[n_random]                 #corresponding label of shuffeled data
        x = x_train[n_random][:]              #random data 
        x = np.reshape(x, (input_dim, input_dim))  #reshape random data

      #creating CNN 
      
      #forward propagation

        Z=conv(x,K)                  #convolving image
        H=Relu(Z)                    #feeding output of conv layer to relu layer
        
        U=np.zeros((output_size,1))  #initailizing 1D array of category
        for i in range(output_size):  #looping through nurens
        	temp1=W[i,:,:,:]            #holding weight w of nuron i
        	temp2=np.multiply(temp1,H)  #perform wx ie weight * x
        	U[i]=np.sum(temp2) + bias[i] #add bias ie Wx+b
        
        rho = softmax_function(U)      #calulate probability of image being in every category
        predicted_value = np.argmax(rho) #select probability with highest score

        if (predicted_value == y):     # if predicted output matches actual output inccrement count varuable
            total_correct += 1

        #backward propagation
        #updating filter weight
        
        diff_U = rho - convert_y(y)    #difference between softmax value  and one hot encoder  i.e calculate bias
        diff_bias = diff_U             #set new bias value
        
        diff_W=np.zeros((output_size,input_dim-filter_size+1,input_dim-filter_size+1,num_filters))  #initialize weight variable
        for i in range(output_size):  #loop through neuron
        	diff_W[i,:,:,:]=diff_U[i]*H  #update new weight


        delta=np.zeros(H.shape)
        for i in range(input_dim-filter_size+1):
        	for j in range(input_dim-filter_size+1):
        		for p in range(num_filters):
       				delta[i,j,p]=np.sum(np.multiply(diff_U,W[:,i,j,p]))

       	grad_Zdel = np.multiply(gradient_Relu(Z),delta)
       	diff_K = conv(x, grad_Zdel)    #new updated filter


       	#parameter updation

       	bias=bias - LR*diff_bias   #updating bias values
       	W = W - LR*diff_W          #updating weights
       	K = K - LR*diff_K          #updating filter

    print("Training accuracy for epoch {} : {}".format(epochs+1, total_correct/np.float(len(x_train))))

#test data
total_correct = 0
for n in range(len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    x = np.reshape(x, (input_dim, input_dim))

    Z=conv(x,K)
    H=Relu(Z)
        
    for i in range(output_size):
       	temp1=W[i,:,:,:]
        temp2=np.multiply(temp1,H)
        U[i]=np.sum(temp2) + bias[i]
        
    rho = softmax_function(U)
    predicted_value = np.argmax(rho)

    if (predicted_value == y):
        total_correct += 1

print("Test accuracy : {}".format(total_correct/np.float(len(x_test))))
